In [34]:
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import gzip
import pickle
import struct

def load_mnist(path, kind='train'):
    import os
    import struct
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        struct.unpack(">IIII", imgpath.read(16))
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels), 784)

    return images, labels


In [40]:
x_train, y_train = load_mnist('fashion', kind='train')
x_test, y_test = load_mnist('fashion', kind='t10k')
idxs = np.random.permutation(x_train.shape[0])
x_train = np.expand_dims(x_train.reshape([-1, 28, 28]), -1)
x_train, x_val = x_train[idxs[:50000]], x_train[idxs[50000:]]
x_test = np.expand_dims(x_test.reshape([-1, 28, 28]), -1)
y_train = keras.utils.np_utils.to_categorical(y_train)
y_train, y_val = y_train[idxs[:50000]], y_train[idxs[50000:]]
y_test = keras.utils.np_utils.to_categorical(y_test)

'''datagen = ImageDataGenerator(
    width_shift_range=0.,
    height_shift_range=0.,
    horizontal_flip=False)
flow = datagen.flow(x_train, y_train, batch_size=128)'''

'datagen = ImageDataGenerator(\n    width_shift_range=0.,\n    height_shift_range=0.,\n    horizontal_flip=False)\nflow = datagen.flow(x_train, y_train, batch_size=128)'

In [45]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (3, 3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1024),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(keras.optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#model.fit_generator(flow, steps_per_epoch = np.ceil(50000/128), validation_data=(x_val, y_val), epochs=5)
model.fit(x_train, y_train, batch_size=128, validation_data=(x_val, y_val), epochs=5)

In [ ]:
model.optimizer.lr = 1e-4
model.fit(x_train, y_train, batch_size=128, validation_data=(x_val, y_val), epochs=3)

In [ ]:
result = model.test_on_batch(x_test, y_test)

In [ ]:
print("test accuracy: {:.2f}".format(100*result[1]))

In [23]:
x_train.shape

(50000, 28, 28, 1)

In [24]:
y_train.shape

(50000, 10)

In [25]:
x_val.shape

(10000, 28, 28, 1)

In [26]:
y_val.shape

(10000, 10)